# Transduction Detection Analysis: Old vs New Methods

This notebook compares the old hardcoded-value transduction detection with the new augmentation-invariance method on the superking dataset.

## Overview
- **Old method**: Detects hardcoded training/test outputs by string matching
- **New method**: Tests augmentation invariance (vertical flip, horizontal flip, color rotation)
- **Dataset**: BigQuery superking dataset (`trelis-arc.arc.superking`)

## Analysis Goals
1. Download all programs and task_ids from superking dataset
2. Apply both transduction detection methods
3. Compare classification differences and analyze patterns
4. Identify cases where methods disagree and understand why

In [1]:
# Setup project path and imports
from pathlib import Path
import sys
project_root = next((parent for parent in [Path.cwd()] + list(Path.cwd().parents) if (parent / "pyproject.toml").exists()), Path.cwd())
sys.path.append(str(project_root))

import pandas as pd
import numpy as np
from google.cloud import bigquery
from typing import Dict, List, Tuple, Any
import json
from tqdm.auto import tqdm

# Import our transduction detection functions
from llm_python.utils.transduction import detect_transduction, detect_transduction_augmentation
from llm_python.utils.arc_tester import ArcTester
from llm_python.utils.task_loader import TaskData
from llm_python.datasets.bigquery_export import load_bigquery_table_as_dataframe

print("✓ All imports successful")
print(f"✓ Project root: {project_root}")

✓ All imports successful
✓ Project root: /home/lewis/code/trelis-arc


/home/lewis/code/trelis-arc/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Step 1: Download superking dataset from Google Cloud Storage
from google.cloud import storage
import glob

# GCS bucket and path
gcs_bucket = "trelis-arc"
gcs_prefix = "datasets/superking"
local_download_dir = "/tmp/superking_data"
output_path = "/tmp/superking_analysis.parquet"

print("Downloading superking dataset from Google Cloud Storage...")
print(f"Bucket: gs://{gcs_bucket}/{gcs_prefix}")

# Create local directory
Path(local_download_dir).mkdir(parents=True, exist_ok=True)

try:
    # Initialize GCS client
    storage_client = storage.Client()
    bucket = storage_client.bucket(gcs_bucket)
    
    # List all parquet files in the superking dataset
    blobs = list(bucket.list_blobs(prefix=gcs_prefix))
    parquet_blobs = [blob for blob in blobs if blob.name.endswith('.parquet')]
    
    print(f"✓ Found {len(parquet_blobs)} parquet files in {gcs_prefix}")
    
    # Download all parquet files
    local_files = []
    for blob in parquet_blobs:
        filename = blob.name.split('/')[-1]  # Get just the filename
        local_path = f"{local_download_dir}/{filename}"
        
        print(f"  Downloading {blob.name} -> {local_path}")
        blob.download_to_filename(local_path)
        local_files.append(local_path)
    
    print(f"✓ Downloaded {len(local_files)} files")
    
    # Combine all parquet files into a single DataFrame
    print("Combining parquet files...")
    dataframes = []
    for file_path in local_files:
        df_part = pd.read_parquet(file_path)
        print(f"  {file_path}: {len(df_part):,} rows")
        dataframes.append(df_part)
    
    # Concatenate all dataframes
    raw_data = pd.concat(dataframes, ignore_index=True)
    
    print(f"✓ Combined dataset: {len(raw_data):,} programs")
    print(f"✓ Columns: {list(raw_data.columns)}")
    
    # Save combined dataset to local parquet for analysis
    raw_data.to_parquet(output_path, index=False)
    print(f"✓ Saved combined dataset to: {output_path}")
    
    # Show sample of the data
    print(f"\nFirst few rows:")
    print(raw_data.head())
    
    # Clean up individual downloaded files
    import os
    for file_path in local_files:
        os.remove(file_path)
    os.rmdir(local_download_dir)
    print(f"✓ Cleaned up temporary files")
    
except Exception as e:
    print(f"❌ Error downloading data: {e}")
    raise

Table: trelis-arc.arc.superking
Exporting BigQuery table 'trelis-arc.arc.superking' to GCS with sharding...
Waiting for BigQuery export to complete...
❌ Error downloading data: 400 trelis-arc:arc.superking is not allowed for this operation because it currently has type EXTERNAL.; reason: invalid, message: trelis-arc:arc.superking is not allowed for this operation because it currently has type EXTERNAL.
Waiting for BigQuery export to complete...
❌ Error downloading data: 400 trelis-arc:arc.superking is not allowed for this operation because it currently has type EXTERNAL.; reason: invalid, message: trelis-arc:arc.superking is not allowed for this operation because it currently has type EXTERNAL.


BadRequest: 400 trelis-arc:arc.superking is not allowed for this operation because it currently has type EXTERNAL.; reason: invalid, message: trelis-arc:arc.superking is not allowed for this operation because it currently has type EXTERNAL.

In [ ]:
# Step 2: Examine data structure and prepare for analysis
df = pd.read_parquet(output_path)

print("Dataset Overview:")
print(f"- Total programs: {len(df):,}")
print(f"- Unique tasks: {df['task_id'].nunique():,}")
print(f"- Columns: {list(df.columns)}")
print(f"- Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Check what columns we have for task data
print(f"\nColumn details:")
for col in df.columns:
    print(f"- {col}: {df[col].dtype}")
    if col in ['predicted_train_output', 'predicted_test_output']:
        print(f"  Sample value type: {type(df[col].iloc[0])}")

# Look at a sample program
print(f"\nSample program:")
sample_row = df.iloc[0]
print(f"Task ID: {sample_row['task_id']}")
print(f"Code length: {len(sample_row['code'])} characters")
print(f"Code preview:\n{sample_row['code'][:200]}...")

# Check if we have task data or need to load it separately
if 'task_data' in df.columns:
    print("✓ Task data is included in the dataset")
else:
    print("⚠️  Task data not included - we'll need to load it separately")

In [ ]:
# Step 3: Load ARC task data using TaskLoader
from llm_python.utils.task_loader import TaskLoader

# Initialize the task loader - this will automatically find and load all ARC data
print("Initializing TaskLoader...")
task_loader = TaskLoader()

print(f"✓ TaskLoader initialized with {len(task_loader.tasks)} tasks available")
print(f"✓ Available subsets: {list(task_loader.subsets.keys())[:10]}...")  # Show first 10

# Check coverage of our dataset
unique_task_ids = set(df['task_id'].unique())
available_tasks = set(task_loader.tasks.keys())
covered_tasks = unique_task_ids.intersection(available_tasks)
missing_tasks = unique_task_ids - available_tasks

print(f"✓ Task coverage: {len(covered_tasks)}/{len(unique_task_ids)} ({100*len(covered_tasks)/len(unique_task_ids):.1f}%)")
if missing_tasks:
    print(f"⚠️  Missing {len(missing_tasks)} tasks: {list(missing_tasks)[:5]}...")

# Filter to only programs with available task data
df_filtered = df[df['task_id'].isin(available_tasks)].copy()
print(f"✓ Filtered to {len(df_filtered)} programs with task data available")

# Helper function to get task data for our analysis
def get_task_data(task_id: str) -> TaskData:
    """Get task data for a given task ID"""
    try:
        return task_loader.get_task(task_id)
    except FileNotFoundError:
        print(f"⚠️  Task {task_id} not found")
        return None

In [ ]:
# Step 4: Run both transduction detection methods on the dataset
# Initialize ArcTester for the new method
arc_tester = ArcTester(timeout=2)

def analyze_program_transduction(row):
    """Apply both old and new transduction detection to a single program"""
    task_id = row['task_id']
    program = row['code']
    
    # Get task data
    task_data = get_task_data(task_id)
    if task_data is None:
        return {
            'old_train_transductive': None,
            'old_train_reason': 'Task not found',
            'old_test_transductive': None, 
            'old_test_reason': 'Task not found',
            'new_transductive': None,
            'new_reason': 'Task not found',
            'error': 'Task not found'
        }
    
    try:
        # Old method: detect_transduction (hardcoded values)
        old_train_transductive, old_train_reason, old_test_transductive, old_test_reason = detect_transduction(
            program, task_data, debug=False
        )
        
        # New method: detect_transduction_augmentation (augmentation invariance)
        new_transductive, new_reason = detect_transduction_augmentation(
            program, task_data, arc_tester, debug=False
        )
        
        return {
            'old_train_transductive': old_train_transductive,
            'old_train_reason': old_train_reason,
            'old_test_transductive': old_test_transductive,
            'old_test_reason': old_test_reason, 
            'new_transductive': new_transductive,
            'new_reason': new_reason,
            'error': None
        }
        
    except Exception as e:
        return {
            'old_train_transductive': None,
            'old_train_reason': f'Error: {str(e)}',
            'old_test_transductive': None,
            'old_test_reason': f'Error: {str(e)}',
            'new_transductive': None,
            'new_reason': f'Error: {str(e)}',
            'error': str(e)
        }

print("Starting transduction analysis...")
print(f"Analyzing {len(df_filtered)} programs...")

# Sample a smaller subset for initial testing (can increase later)
sample_size = min(1000, len(df_filtered))  # Start with 1000 programs
df_sample = df_filtered.sample(n=sample_size, random_state=42).copy()
print(f"Using sample of {len(df_sample)} programs for analysis")

# Apply analysis to each program
tqdm.pandas(desc="Analyzing programs")
results = df_sample.progress_apply(analyze_program_transduction, axis=1, result_type='expand')

# Combine results with original data
df_analysis = pd.concat([df_sample.reset_index(drop=True), results], axis=1)

print(f"✓ Analysis complete for {len(df_analysis)} programs")
print(f"✓ Success rate: {(df_analysis['error'].isna()).sum()}/{len(df_analysis)} ({100*(df_analysis['error'].isna()).sum()/len(df_analysis):.1f}%)")

# Clean up ArcTester
ArcTester.cleanup_executor()

In [ ]:
# Step 5: Analyze differences between old and new methods
import matplotlib.pyplot as plt
import seaborn as sns

# Filter out error cases for analysis
df_valid = df_analysis[df_analysis['error'].isna()].copy()
print(f"Valid results: {len(df_valid)}/{len(df_analysis)} programs")

# Create comparison categories
df_valid['old_any_transductive'] = df_valid['old_train_transductive'] | df_valid['old_test_transductive'] 
df_valid['agreement'] = df_valid['old_any_transductive'] == df_valid['new_transductive']

# Summary statistics
print("\n" + "="*60)
print("TRANSDUCTION DETECTION COMPARISON")
print("="*60)

print(f"\nOLD METHOD (Hardcoded Detection):")
print(f"  - Train transductive: {df_valid['old_train_transductive'].sum():,} ({100*df_valid['old_train_transductive'].mean():.1f}%)")
print(f"  - Test transductive: {df_valid['old_test_transductive'].sum():,} ({100*df_valid['old_test_transductive'].mean():.1f}%)")
print(f"  - Any transductive: {df_valid['old_any_transductive'].sum():,} ({100*df_valid['old_any_transductive'].mean():.1f}%)")

print(f"\nNEW METHOD (Augmentation Invariance):")
print(f"  - Transductive: {df_valid['new_transductive'].sum():,} ({100*df_valid['new_transductive'].mean():.1f}%)")

print(f"\nAGREEMENT:")
agreement_rate = df_valid['agreement'].mean()
print(f"  - Methods agree: {df_valid['agreement'].sum():,}/{len(df_valid)} ({100*agreement_rate:.1f}%)")

# Disagreement analysis
disagreement = df_valid[~df_valid['agreement']].copy()
print(f"  - Methods disagree: {len(disagreement):,} ({100*(1-agreement_rate):.1f}%)")

if len(disagreement) > 0:
    old_yes_new_no = disagreement[disagreement['old_any_transductive'] & ~disagreement['new_transductive']]
    old_no_new_yes = disagreement[~disagreement['old_any_transductive'] & disagreement['new_transductive']]
    
    print(f"    • Old=Transductive, New=Non-transductive: {len(old_yes_new_no):,}")
    print(f"    • Old=Non-transductive, New=Transductive: {len(old_no_new_yes):,}")

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Overall comparison
agreement_counts = df_valid['agreement'].value_counts()
axes[0,0].pie(agreement_counts.values, labels=['Disagree', 'Agree'], autopct='%1.1f%%', startangle=90)
axes[0,0].set_title('Method Agreement Rate')

# 2. Transduction rates by method
methods = ['Old (Any)', 'New (Augmentation)']
rates = [df_valid['old_any_transductive'].mean(), df_valid['new_transductive'].mean()]
axes[0,1].bar(methods, rates, color=['skyblue', 'lightcoral'])
axes[0,1].set_ylabel('Transduction Rate')
axes[0,1].set_title('Transduction Detection Rates')
axes[0,1].set_ylim(0, 1)

# 3. Disagreement breakdown
if len(disagreement) > 0:
    disagreement_types = ['Old=T, New=NT', 'Old=NT, New=T']
    disagreement_counts = [len(old_yes_new_no), len(old_no_new_yes)]
    axes[1,0].bar(disagreement_types, disagreement_counts, color=['orange', 'purple'])
    axes[1,0].set_ylabel('Number of Programs')
    axes[1,0].set_title('Types of Disagreement')
    axes[1,0].tick_params(axis='x', rotation=45)

# 4. Task distribution
task_counts = df_valid['task_id'].value_counts().head(20)
axes[1,1].bar(range(len(task_counts)), task_counts.values)
axes[1,1].set_xlabel('Top 20 Tasks (by program count)')
axes[1,1].set_ylabel('Number of Programs')
axes[1,1].set_title('Programs per Task Distribution')

plt.tight_layout()
plt.show()

print(f"\n✓ Analysis visualization complete")

In [ ]:
# Step 6: Examine specific disagreement cases
print("="*80)
print("DETAILED DISAGREEMENT ANALYSIS")
print("="*80)

if len(disagreement) > 0:
    print(f"\nAnalyzing {len(disagreement)} disagreement cases...")
    
    # Case 1: Old method says transductive, new method says non-transductive
    if len(old_yes_new_no) > 0:
        print(f"\n🔍 CASE 1: Old=Transductive, New=Non-transductive ({len(old_yes_new_no)} cases)")
        print("These programs have hardcoded values but show augmentation invariance")
        print("-" * 60)
        
        # Show a few examples
        for i, (idx, row) in enumerate(old_yes_new_no.head(3).iterrows()):
            print(f"\nExample {i+1}:")
            print(f"Task ID: {row['task_id']}")
            print(f"Old train reason: {row['old_train_reason']}")
            print(f"Old test reason: {row['old_test_reason']}")
            print(f"New reason: {row['new_reason']}")
            print(f"Code (first 200 chars):")
            print(row['code'][:200] + "..." if len(row['code']) > 200 else row['code'])
            print("-" * 40)
    
    # Case 2: Old method says non-transductive, new method says transductive  
    if len(old_no_new_yes) > 0:
        print(f"\n🔍 CASE 2: Old=Non-transductive, New=Transductive ({len(old_no_new_yes)} cases)")
        print("These programs don't have obvious hardcoded values but fail augmentation invariance")
        print("-" * 60)
        
        # Show a few examples
        for i, (idx, row) in enumerate(old_no_new_yes.head(3).iterrows()):
            print(f"\nExample {i+1}:")
            print(f"Task ID: {row['task_id']}")
            print(f"Old train reason: {row['old_train_reason']}")
            print(f"Old test reason: {row['old_test_reason']}")
            print(f"New reason: {row['new_reason']}")
            print(f"Code (first 200 chars):")
            print(row['code'][:200] + "..." if len(row['code']) > 200 else row['code'])
            print("-" * 40)

# Save results for further analysis
output_file = "/tmp/transduction_analysis_results.parquet"
df_analysis.to_parquet(output_file, index=False)
print(f"\n✓ Results saved to: {output_file}")

print(f"\n✅ ANALYSIS COMPLETE")
print(f"📊 Analyzed {len(df_valid)} valid programs")
print(f"🤝 Agreement rate: {100*agreement_rate:.1f}%")
print(f"📁 Results saved for further investigation")